# Valutazione

1. Creare tot sistemi diversi (per esempio, lucene, bert, scibert, tabert (con e senza contesto))
2. Creare un sottoinsieme dei paper (tipo 20) da usare come ground truth (o a caso oppure con lucene i più rilevanti per argomento che abbiano tabelle interessanti) - cerchiamo di limitare il numero di tabelle a ca. 50
3. Per ogni query $q \in Q$ (min 5):
    1. Fare il ranking a mano delle tabelle
        - Salviamo i ranking per ogni query in un json, con le informazioni rilevanti, tipo il ranking, il valore di rilevanza per ogni elemento etc.
    2. Interrogare ogni sistema sulla query
    3. Calcolare le metriche: 
        - Reciprocal Rank: $\text{RR}_q = \frac{1}{rank_i}$ dove $i$ è l’elemento più rilevante.
            - nella pratica possiamo controllare se l’elemento scelto dal motore ha almeno lo score massimo (potrebbero esserci dei parimerito)
        - Normalized Discounted Cumulative Gain con taglio $\text{K} = \set{5,15}$:
            
            $$
            \text{NDCG@K}_q = \frac{\text{DCG@K}_q}{\text{IDCG@K}_q}
            $$
            
            - dove dividiamo il $\text{DCG@K}_q = rel_1 + \sum_{i=2}^K \frac{rel_i}{\log_2 (i + 1)}$ con quello ideale, cioè dove il ranking è il migliore possibile
4. Calcolare la media delle metriche:
    - Mean Reciprocal Rank: $\text{MRR} = \frac{1}{|Q|} \sum_{q \in Q} \text{RR}_q$
    - Media dei NDCG: $\frac{1}{|Q|} \sum_{q \in Q} \text{NDCG@K}_q$

### Query (in verde stesso ranking ma proviamo sinonimi)

1. NDCG su dataset movielens ✅
2. Recommender systems Recall su dataset goodbook ✅
3. Recommender systems MRR ✅
4. Deep Learning dataset Apple Flower ✅
5. Deep Learning GPT3 precision f1 ✅
6. Deep Learning GPT3 precision f-measure ✅

In [7]:
import json

results_file = "./results.json"
ground_truth_path = "./ground_truth"
num_queries = 6

# model -> method -> query -> (position, id) 
results: dict[str, dict[str, dict[str, dict[str, str]]]] = {}

# query -> (position, id) 
ground_truth: dict[str, dict[str, str]] = {}

with open(results_file, "r", encoding="utf-8") as file:
    results = json.load(file)
    
for i in range(1, num_queries + 1):
    query_id = f"q{i}"
    with open(ground_truth_path + f"/{query_id}_rank.json", "r", encoding="utf-8") as file:
        ground_truth[query_id] = json.load(file)
            

## MRR

In [21]:
mrr_values: dict[str, dict[str, float]] = {}

for model, methods in results.items():
    mrr_values[model] = {}
    for method, queries in methods.items():
        sum_rr = 0
        not_founds = 0
        
        for query_id, ranking in queries.items():
            best_table: str = ground_truth[query_id]["1"]
            best_table_id = best_table["paper_id"] + "#" + best_table["table_id"]
            
            rr = 0
            for pos, table_id in ranking.items():
                if (table_id == best_table_id): rr = 1.0 / float(pos)
            
            if rr == 0: not_founds += 1
            sum_rr += rr
            
            
        
        mrr = sum_rr / num_queries
        mrr_values[model][method] = mrr
        print(f"MRR value for {model} using method: {method} is {mrr} --- best not found in {not_founds}/{num_queries} queries.")
        

MRR value for lucene using method: bm25 is 0.041666666666666664 --- best not found in 5/6 queries.
MRR value for allenai/scibert_scivocab_uncased using method: tab_embedding is 0.041666666666666664 --- best not found in 5/6 queries.
MRR value for allenai/scibert_scivocab_uncased using method: tab_cap_embedding is 0.015151515151515152 --- best not found in 5/6 queries.
MRR value for allenai/scibert_scivocab_uncased using method: tab_cap_ref_embedding is 0.0 --- best not found in 6/6 queries.
MRR value for allenai/scibert_scivocab_uncased using method: weighted_embedding is 0.023809523809523808 --- best not found in 5/6 queries.
MRR value for bert-base-uncased using method: tab_embedding is 0.020833333333333332 --- best not found in 5/6 queries.
MRR value for bert-base-uncased using method: tab_cap_embedding is 0.013888888888888888 --- best not found in 5/6 queries.
MRR value for bert-base-uncased using method: tab_cap_ref_embedding is 0.0 --- best not found in 6/6 queries.
MRR value for